## Problem Statement

We wish to learn a generic, continuous system $\mathbf f$ with a state space $\mathbf x \in \mathcal X \subseteq \mathbf R^N$ and an input space $\mathbf u \in \mathcal U \subseteq \mathbf R^M$, 

\begin{equation}
\dot{\mathbf x} = \mathbf f(\mathbf x, \mathbf u),
\end{equation}

from a set of trajectories $\mathcal T = \{[(\mathbf x_{11}, \mathbf u_{11}), \dots, (\mathbf x_{n_11}, \mathbf u_{n_11})], \dots, [(\mathbf x_{1m}, \mathbf u_{1m}), \dots, (\mathbf x_{n_mm}, \mathbf u_{n_mm})]\}$ where each trajectory in $t_i \in \mathcal T$ has an arbitrary length ${n_i}$. We find the system state after some time $\tau$ by solving the initial value problem on the initial state $\mathbf x_0$.


### Koopman Invariant Subspace

Rather than learn $\mathbf f$ generically, we wish to use *Koopman Linearization*, a globally linear representation of the system. Koopman linearization utilizes a new space, the image of the states under an *observables* function $\mathbf g: \mathcal X \rightarrow \mathcal H$, where the dynamics are governed by a linear operator $\mathcal K$, the so-called *Koopman Operator*. This space $\mathcal H$ is a Hilbert space with typically infinite dimensions, making the approximation of the Koopman operator challenging. Usually, the goal is relaxed to finding a finite dimensional operator that acts on a subspace where the dynamics are *weakly linear*, recovering a *Koopman invariant subspace*. It is worth noting that techniques exist which do recover an infinite dimensional linear operator, often using things like the *kernel trick*, but we value having access to the operator explicitely.

## Deep Learning Approach

### Baseline Formulation

For any number of trajectories, we can construct the matrices $\mathbf X_1$, $\mathbf X_2$ such that $\mathbf F_{\Delta t}(\mathbf X_1) = \mathbf X_2$, where $\mathbf F_t$ evolves the state $\mathbf x_0$ $t$ time into the future. Given the given Koopman linearization, this motivates the optimization problem

\begin{equation}
\operatorname*{argmin}_{\mathbf g, \mathcal K} \| \mathbf g(\mathbf X_2) - \mathcal K_{\Delta t} \mathbf g(\mathbf X_1) \|_{\text{MSE}},
\end{equation}

noting that setting $\mathbf g(\mathbf x) = \mathbf x$ changes the problem to dynamic mode decomposition (DMD) and setting $\mathbf g(\mathbf x) = \mathbf f(\mathbf x)$ changes the problem to extended dynamic mode decomposition (eDMD). 


We formulate a network $G_\Theta (\mathbf x) = (\mathbf g_{\theta}^{-1} \circ \mathcal K_{\theta} \circ \mathbf g_{\theta}) (\mathbf x)$ where $\mathbf g$, $\mathbf g^{-1}$ are a encoder/decoder pair and $\mathcal K$ is a linear layer with no bias (matching the Koopman operator definition). Note that we have some hyperparameters $\Theta$ that we get from typical neural network constructions. Unlike common autoencoder architectures, the latent space will likely have higher dimensionality than the original space so that the dynamics are approximately linear. Accordingly, a loss will needed to be added to account for how well the encoder embedds linear dynamics. In total, we implement the following loss functions

* *Reconstruction Loss*: $\mathcal L_{\text{recon}}=\left\| \mathbf x_1 - \mathbf g^{-1}(\mathbf g (\mathbf x_1)) \right\|_{\text{MSE}}$ Standard autoencoder loss.
* *Prediction Loss*: $\mathcal L_{\text{pred}}=\left\| \mathbf x_2 - \mathbf g^{-1}(\mathcal K \mathbf g (\mathbf x_1)) \right\|_{\text{MSE}}$ How well network can predict dynamics.
* *Linearity Loss*: $\mathcal L_{\text{lin}}=\left\| \mathbf g(x_2) - \mathcal K \mathbf g (\mathbf x_1) \right\|_{\text{MSE}}$ How well the network can predict dynamics in the observables space (Koopman invariant subspace).
* *Largest Loss*: $\mathcal L_{\infty}=\left\| \mathbf x_1 - \mathbf g^{-1}(\mathbf g (\mathbf x_1)) \right\|_{\infty}$ + $\left\| \mathbf x_2 - \mathbf g^{-1}(\mathcal K \mathbf g (\mathbf x_1)) \right\|_{\infty}$ Loss for maximum deviation in state or observables space.
* *Metric Loss* $\mathcal L_{\text{metric}}=\left| \left\| \mathbf g(x_2) - \mathbf g(x_1) \right\|_{\text{MSE}} - \left\| x_2 - x_1 \right\|_{\text{MSE}} \right|$ Encourage distances to be preserved in the observables space--useful for inputs that will be directly injected into observables space.

for the loss function

\begin{equation}
\mathcal L = \alpha_1 \left(\mathcal L_{\text{recon}} + \mathcal L_{\text{pred}}\right) + \mathcal L_{\text{lin}} + \alpha_2 \mathcal L_{\infty} + \alpha_3 \mathcal L_{\text{metric}} + \alpha_4 \| \mathbf W \|_2^2,
\end{equation}

where $\mathbf W$ are the weights of the hidden layers of the form $\mathbf W \mathbf x + \mathbf b$ with rectified linear units (ReLU).

### Extensions to Systems with Inputs

For systems with inputs, the network $G'_\Theta (\mathbf x, \mathbf u) = \left( \mathbf g_{\theta}^{-1} \circ \begin{bmatrix} \mathcal K_{\theta} && \mathbf B_{\theta}\end{bmatrix} \circ \begin{bmatrix} \mathbf g_\theta (.)\\ \mathbf u \end{bmatrix} \right) (\mathbf x)$ suffices with the new linear operator $\mathbf B_\theta$. In the cost function, *metric loss* is added to the inputs can be injected in a space that approximately preserves the distance of the state space, where it is are added for linear systems.


### Dimensionality Reduction in the Latent Space

As commonly seen in perturbation theory, nonlinearities generate higher harmonics, and so an auxiliary network can be employed to identify the continuous eigenvalue spectrum $G'_\Theta (\mathbf x) = (\mathbf g_{\theta}^{-1} \circ \mathcal K_{\theta}(\mathbf \lambda (\mathbf g_\theta (\mathbf x))) \circ \mathbf g_{\theta}) (\mathbf x)$ . Further, if this explicit frequency dependence is unaccounted for, then a high-dimensional network is necessary to account for the shifting frequency and eigenvalues. 



### State Projection

We can trivially implement the decoder by restricting the observables to include the states $G'_\Theta (\mathbf x) = \left(\begin{bmatrix} \mathbf 1_{N} && \mathbf 0\end{bmatrix} \circ \mathcal K_{\theta} \circ \begin{bmatrix} \mathbf x \\ \mathbf g_\theta (.)\end{bmatrix} \right) (\mathbf x)$. This is useful for set propagation-based reachability, where the ability to project sets into the original state space is helped by this relation. Propagating a set of some known representation has been of interest, and work like ReLUplex shows that an overapproximation of an image of some initial set under a neural network using ReLU nonlinearities can be done efficiently.

## References

Lusch, B., Kutz, J. N., & Brunton, S. L. (2018). Deep learning for universal linear embeddings of nonlinear dynamics. Nature communications, 9(1), 1-10.

Li, Y., He, H., Wu, J., Katabi, D., & Torralba, A. (2019). Learning compositional koopman operators for model-based control. arXiv preprint arXiv:1910.08264.

Katz, G., Barrett, C., Dill, D. L., Julian, K., & Kochenderfer, M. J. (2017, July). Reluplex: An efficient SMT solver for verifying deep neural networks. In International conference on computer aided verification (pp. 97-117). Springer, Cham.

In [ ]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("..")
# this is the convenience function
from autokoopman import auto_koopman

In [ ]:
# for a complete example, let's create an example dataset using an included benchmark system
import autokoopman.benchmark.fhn as fhn
dt = 0.1
tspan = 10.0
fhn = fhn.FitzHughNagumo()
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.5, high=2.5, size=(50, 2)),
    tspan=[0.0, dt*5],
    sampling_period=dt
)

test_ivs = np.random.uniform(low=-2.0, high=2.0, size=(20, 2))
test_data = fhn.solve_ivps(
    initial_states=test_ivs,
    tspan=[0.0, tspan],
    sampling_period=dt
)

In [ ]:
import autokoopman.estimator.deepkoopman as dk

koop = dk.DeepKoopman(
    state_dim=len(training_data.state_names), 
    input_dim=0, 
    hidden_dim=12, 
    max_iter=2000,
    lr=1e-3, 
    hidden_enc_dim=64,
    num_hidden_layers=2,  
    metric_loss_weight=1e-2, 
    pred_loss_weight=1e-2, 
    validation_data=test_data
)

In [ ]:
# do training
koop.fit(training_data)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(koop.loss_hist['validation_lin_loss'], label="linearity loss")
plt.plot(koop.loss_hist['validation_pred_loss'], label="prediction loss")
plt.plot(koop.loss_hist['validation_total_loss'], label="total loss")
#plt.plot(koop.loss_hist['total_loss'], label="total loss")
#plt.plot(koop.loss_hist['validation_metric_loss'], label="metric loss")
#plt.plot(koop.loss_hist['validation_recon_loss'], label="reconstruction loss")

#plt.plot(koop.loss_hist['lin_loss'], label="linearity loss")
#plt.plot(koop.loss_hist['pred_loss'], label="prediction loss")
#plt.plot(koop.loss_hist['total_loss'], label="total loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.yscale("log")
plt.title("Deep Learning Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# solve for test trajectories
fit_data = koop.model.solve_ivps(test_ivs, tspan=[0.0, tspan], sampling_period=dt)

In [ ]:
plt.figure(figsize=(8, 8))
for t in training_data:
    plt.plot(*t.states.T, 'gray', alpha=0.3)
for t, tt in zip(fit_data, test_data):
    plt.plot(*t.states.T, 'r')
    plt.plot(*tt.states.T, 'k', alpha=0.5)
    
plt.title("FHN Deep Koopman")